In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, Input, Flatten, concatenate,LSTM,Dropout
from tensorflow.keras.models import Model
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
def ncf_model(arch):
  user_input = Input(shape=(1,), name = 'user_input')
  embed_user = Embedding(input_dim=nb_users, output_dim=64, input_length=1)(user_input)
  embed_user = Flatten()(embed_user)


  hotel_input = Input(shape=(1,), name = 'item_input')
  embed_hotel = Embedding(input_dim=nb_hotels, output_dim=64, input_length=1)(hotel_input)
  embed_hotel = Flatten()(embed_hotel)

  layers0 = concatenate([embed_user,embed_hotel])

  if len(arch)>1:

    hidden = Dense(arch[0],activation='relu')(layers0)

    for j in range(1,len(arch)):
      hidden = Dense(arch[j],activation='relu')(hidden)
  else:
    hidden = Dense(8,activation='relu')(layers0) 

  out = Dense(5)(hidden)
  model = Model(inputs=[user_input, hotel_input], 
                  outputs=out)
  
  model.compile(optimizer='adam',loss='mae',metrics=['mae'])

  return model

In [ ]:
df = pd.DataFrame(data={'alpha':[],'beta':[],	'mean mae':[],	'mae_cleanliness':[],
                        'mae_location':[],'mae_rooms':[],'mae_service':[],'mae_sleep':[],'mae_value':[],	
                        'mean rmse':[], 'rmse_cleanliness':[],'rmse_location':[],'rmse_rooms':[],	
                        'rmse_service':[],'rmse_sleep':[],'rmse_value':[]})
df

,alpha,beta,mean mae,mae_cleanliness,mae_location,mae_rooms,mae_service,mae_sleep,mae_value,mean rmse,rmse_cleanliness,rmse_location,rmse_rooms,rmse_service,rmse_sleep,rmse_value


In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from sklearn.neighbors import DistanceMetric
kf = KFold(n_splits=5,random_state=24,shuffle=True)
from sklearn.metrics.pairwise import cosine_similarity


dist = DistanceMetric.get_metric('hamming')




value_alpha = [0.7,0.5,0.3]
dic_alpha = {}
for alpha in value_alpha:

        dic_alpha[alpha] = {}
        dic_alpha[alpha]['mae_clean'] = []
        dic_alpha[alpha]['mae_location'] = []
        dic_alpha[alpha]['mae_rooms'] = []
        dic_alpha[alpha]['mae_service'] = []
        dic_alpha[alpha]['mae_value'] = []
        dic_alpha[alpha]['mae_mean'] = []

        dic_alpha[alpha]['rmse_clean'] = []
        dic_alpha[alpha]['rmse_location'] = []
        dic_alpha[alpha]['rmse_rooms'] = []
        dic_alpha[alpha]['rmse_service'] = []
        dic_alpha[alpha]['rmse_value'] = []
        dic_alpha[alpha]['rmse_mean'] = []
k=7
for i in range(1,6):
    pred_Bert_cf = []
    train_data = pd.read_csv('/content/drive/MyDrive/Trip_multi_fold_with_sentiment'+str(i)+'_train.csv')
    test_data = pd.read_csv('/content/drive/MyDrive/Trip_multi_fold_with_sentiment'+str(i)+'_test.csv')
    train_data.shape , test_data.shape
    nb_users = int(max(train_data.user_id.unique())) +1
    nb_hotels = int(max(train_data.offering_id.unique())) +1
    nb_users, nb_hotels

    ncf = ncf_model([128,64,32,16,8])

    y = train_data[['cleanliness','location','rooms','service','value']].values
    y_test = test_data[['cleanliness','location','rooms','service','value']].values
    callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    ncf.fit([train_data.user_id.values,train_data.offering_id.values],y,validation_split=0.1,callbacks=[callback],
              
              epochs=20,batch_size=128)
    ncf_pred = ncf.predict([test_data.user_id.values,test_data.offering_id.values])
    cmp=0
    for idx,row in test_data.iterrows():
      cmp+=1
      precedent_data = train_data[train_data['user_id']==row['user_id']]
      precedent_hotels = precedent_data.offering_id.values
      prcd_reviews = precedent_data[['cleanliness_sent','location_sent','rooms_sent','service_sent',
                                         'value_sent']].values
      hotel_id = row['offering_id']
      
      if hotel_id in train_data.offering_id.values:
        precedent_data = train_data[train_data['user_id']==row['user_id']]

        if precedent_data.shape[0] > 0:

          precedent_hotels = precedent_data.offering_id.values
          prcd_reviews = precedent_data[['cleanliness_sent','location_sent','rooms_sent','service_sent',
                                         'value_sent']].values
          
        
          prcedent_data_sentiment_review = {}
          for h_id,rev in zip(precedent_hotels,prcd_reviews):
            
            
            prcedent_data_sentiment_review[h_id] = rev.mean()
            
          
          user = row['user_id']

          maybe_sim_users = []
          sim = {}
          if cmp%100==0:
            print('rest',test_data.shape[0] - cmp)

          users = train_data[train_data['offering_id']==hotel_id].user_id.unique()
          for user1 in users:
            if (user1 != user):
              

              user1_data = train_data[train_data['user_id']==user1]
              sim_items = set(precedent_hotels).intersection(set(user1_data.offering_id.values))
              if len(sim_items) > 0:
                  
                  maybe_sim_users.append(user1)
                
                  user_rat = []
                  user1_rat = []
                  for item in sim_items:
                    
                    user_rat.append(prcedent_data_sentiment_review[item])

                  
                    rev = user1_data[user1_data['offering_id']==item][['cleanliness_sent','location_sent','rooms_sent','service_sent',
                                         'value_sent']].values
                    
                    
                    user1_rat.append(rev[0].mean())
                  
                  if  1 - abs(cosine_similarity([user_rat],[user1_rat])[0][0]) > 0.5:        
                    sim[user1] = 1 - abs(cosine_similarity([user_rat],[user1_rat])[0][0])
          
          if len(sim.keys())>0: #similar users
            rat = [0.0,0.0,0.0,0.0,0.0]
            sum_sim = 0.0
            for user1 in list(sim.keys())[:k]:
              rat += sim[user1] * train_data[(train_data['user_id']==user1) & (train_data['offering_id']==hotel_id ) ][['cleanliness','location','rooms','service','value']].values[0]
              sum_sim+= sim[user1]
            rat = rat / sum_sim
          
          
          else : #no similar users
            rat = [0.0,0.0,0.0,0.0,0.0]
            for i,col in zip([0,1,2,3,4],['cleanliness','location','rooms','service','value']):
              rat[i] = np.mean(train_data[train_data['offering_id']==hotel_id][[col]].values)
          
        else: #new_user
            rat = [0.0,0.0,0.0,0.0,0.0]
            for i,col in zip([0,1,2,3,4],['cleanliness','location','rooms','service','value']):
              rat[i] = np.mean(train_data[train_data['offering_id']==hotel_id][[col]].values)  

      elif train_data[train_data['user_id']==row['user_id']].shape[0]>0: #new_item
        rat = [0.0,0.0,0.0,0.0,0.0]
        for i,col in zip([0,1,2,3,4],['cleanliness','location','rooms','service','value']):
            rat[i] = np.mean(precedent_data[[col]].values)
      else: #new_user , new_item
        #rnd_petubation = random.uniform(-1,1)
        #val = rnd_petubation + 3.0
        rat = [3.0,3.0,3.0,3.0,3.0]
      pred_Bert_cf.append(rat.copy())

    pred_Bert_cf = np.array(pred_Bert_cf)

    
    for alpha in value_alpha:
      pred = alpha*ncf_pred + (1-alpha)*pred_Bert_cf
      pred = np.array(pred)
      dic_alpha[alpha]['mae_clean'].append(mae(pred[:,0],y_test[:,0]))
      dic_alpha[alpha]['mae_location'].append(mae(pred[:,1],y_test[:,1]))
      dic_alpha[alpha]['mae_rooms'].append(mae(pred[:,2],y_test[:,2]))
      dic_alpha[alpha]['mae_service'].append(mae(pred[:,3],y_test[:,3]))
      dic_alpha[alpha]['mae_value'].append(mae(pred[:,4],y_test[:,4]))
      dic_alpha[alpha]['mae_mean'].append(mae(pred,y_test))

      dic_alpha[alpha]['rmse_clean'].append(mse(pred[:,0],y_test[:,0],squared=False))
      dic_alpha[alpha]['rmse_location'].append(mse(pred[:,1],y_test[:,1],squared=False))
      dic_alpha[alpha]['rmse_rooms'].append(mse(pred[:,2],y_test[:,2],squared=False))
      dic_alpha[alpha]['rmse_service'].append(mse(pred[:,3],y_test[:,3],squared=False))
      dic_alpha[alpha]['rmse_value'].append(mse(pred[:,4],y_test[:,4],squared=False))
      dic_alpha[alpha]['rmse_mean'].append(mse(pred,y_test,squared=False))
      

 
      
    


    




/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_distance_metric.py:14: FutureWarning: sklearn.neighbors.DistanceMetric has been moved to sklearn.metrics.DistanceMetric in 1.0. This import path will be removed in 1.3
  category=FutureWarning,


Epoch 1/20
55/55 [==============================] - 2s 15ms/step - loss: 3.6253 - mae: 3.6253 - val_loss: 1.7331 - val_mae: 1.7331
Epoch 2/20
55/55 [==============================] - 0s 9ms/step - loss: 1.2168 - mae: 1.2168 - val_loss: 1.1014 - val_mae: 1.1014
Epoch 3/20
55/55 [==============================] - 0s 8ms/step - loss: 0.7462 - mae: 0.7462 - val_loss: 1.0335 - val_mae: 1.0335
Epoch 4/20
55/55 [==============================] - 0s 7ms/step - loss: 0.6979 - mae: 0.6979 - val_loss: 0.9948 - val_mae: 0.9948
Epoch 5/20
55/55 [==============================] - 0s 6ms/step - loss: 0.6774 - mae: 0.6774 - val_loss: 0.9973 - val_mae: 0.9973
Epoch 6/20
55/55 [==============================] - 0s 6ms/step - loss: 0.6641 - mae: 0.6641 - val_loss: 1.0075 - val_mae: 1.0075
Epoch 7/20
55/55 [==============================] - 0s 6ms/step - loss: 0.6561 - mae: 0.6561 - val_loss: 0.9948 - val_mae: 0.9948
Epoch 8/20
55/55 [==============================] - 0s 6ms/step - loss: 0.6470 - mae: 0.6

InvalidArgumentError: ignored

In [ ]:
dic_alpha

{0.3: {'mae_clean': [0.7592796039544036,
   0.7715815132278522,
   0.7488192324487385,
   0.7314523608935728,
   0.7754853478310755],
  'mae_location': [0.654607765078873,
   0.6505162640413515,
   0.6209565010081052,
   0.6350054830174534,
   0.6471272834233363],
  'mae_mean': [0.8217695125886282,
   0.8105492172326318,
   0.816232960354004,
   0.8084991425328752,
   0.8263237095252874],
  'mae_rooms': [0.8636296602259124,
   0.8478529334573639,
   0.8765217153829854,
   0.8679693939438262,
   0.8531290027464798],
  'mae_service': [0.8793563814648586,
   0.8577878116439701,
   0.8900524821443818,
   0.8641307332358156,
   0.8974968771593379],
  'mae_value': [0.9519741522190914,
   0.9250075637926174,
   0.9448148707858071,
   0.9439377415737085,
   0.9583800364662052],
  'rmse_clean': [1.0483973870114602,
   1.072835839788409,
   1.0314428120287662,
   1.0144309175391124,
   1.0643927375754971],
  'rmse_location': [0.9091535654563236,
   0.9110028878112877,
   0.8893965620224181,
   0

In [ ]:
dic_final = {}
for alpha in value_alpha:
  dic_final[alpha] = {}
  for key in dic_alpha[alpha].keys():
    dic_final[alpha][key] = np.mean(dic_alpha[alpha][key])

In [ ]:
dic_final

{0.3: {'mae_clean': 0.7573236116711286,
  'mae_location': 0.6416426593138239,
  'mae_mean': 0.8166749084466852,
  'mae_rooms': 0.8618205411513135,
  'mae_service': 0.8777648571296727,
  'mae_value': 0.9448228729674859,
  'rmse_clean': 1.0462999387886491,
  'rmse_location': 0.9019071089535735,
  'rmse_mean': 1.0967950984717714,
  'rmse_rooms': 1.1265091611693498,
  'rmse_service': 1.1905265046516427,
  'rmse_value': 1.2187327787956446},
 0.5: {'mae_clean': 0.756549656425,
  'mae_location': 0.6477247050974867,
  'mae_mean': 0.8141286522887972,
  'mae_rooms': 0.8622012466615919,
  'mae_service': 0.8699692400130061,
  'mae_value': 0.9341984132469001,
  'rmse_clean': 1.0437042552401468,
  'rmse_location': 0.902313906454397,
  'rmse_mean': 1.091789860017184,
  'rmse_rooms': 1.1246639671918817,
  'rmse_service': 1.18065926004379,
  'rmse_value': 1.2076079111557216},
 0.7: {'mae_clean': 0.7625697273538278,
  'mae_location': 0.6608655535701998,
  'mae_mean': 0.8194867694047787,
  'mae_rooms': 0

In [ ]:
import json
with open("/content/drive/MyDrive/multi_NCF+BERT_cf_with_callback.json", "w") as outfile:
    json.dump(dic_final, outfile)

In [ ]:
!

In [ ]:
for j in range(1,6):

#Alpha 0.7 beta 0.3

In [ ]:
pred = 0.7*ncf_pred + 0.3*pred_Bert_cf
mae(pred,y_test)

0.8222287391593394

#Alpha 0.5 beta 0.5

In [ ]:
pred = 0.5*ncf_pred + 0.5*pred_Bert_cf
mae(pred,y_test)

0.8178455189825333

#Alpha 0.3 beta 0.7

In [ ]:
pred = 0.3*ncf_pred + 0.7*pred_Bert_cf
mae(pred,y_test)

0.8234194200785353